https://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzMw==&mid=2650734471&idx=1&sn=be4cd4b85ed84f997baf4c88543dc3f4&chksm=871b3bf9b06cb2ef94ea9531ec74fef14b8db5d1996b0cf0c9bd31ca3594ef1f54feaea17109&scene=21#wechat_redirect

In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
var = tf.Variable(initial_value=0.0)

In [3]:
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(tf.global_variables_initializer())
sess2.run(tf.global_variables_initializer())

In [4]:
print("Initial value of var in session 1:", sess1.run(var))
print("Initial value of var in session 2:", sess2.run(var))

Initial value of var in session 1: 0.0
Initial value of var in session 2: 0.0


In [5]:
sess1.run(var.assign_add(1.0))
print("Incremented var in session 1")
print("Value of var in session 1:", sess1.run(var))
print("Value of var in session 2:", sess2.run(var))

Incremented var in session 1
Value of var in session 1: 1.0
Value of var in session 2: 0.0


In [6]:
# 创建一个最简单的TensorFlow集群
c = tf.constant('Hello,Distributed TensorFlow!')
# 创建一个本地的TensorFlow集群
server = tf.train.Server.create_local_server()
# 在集群上创建一个会话
sess = tf.Session(server.target)
print(sess.run(c))

b'Hello,Distributed TensorFlow!'


#### 分布式TensorFlow

In [7]:
# 第一步是定义集群的规模
tasks = ['localhost:2222','localhost:2223']
# 每个任务都与job相关联,该工作是相关任务的集合
jobs = {'local':tasks}
# 所有这些定义为一个集群
cluster = tf.train.ClusterSpec(jobs)
# 指定每个服务器对应为集群定义中的哪个服务器
# "This server corresponds to the the first task (task_index=0)
# of the tasks associated with the 'local' job."
server1 = tf.train.Server(cluster, job_name="local", task_index=0)
server2 = tf.train.Server(cluster, job_name="local", task_index=1)

In [8]:
# 所有的服务器上运行同一个静态计算图，且每个图只有一个变量
tf.reset_default_graph()
var = tf.Variable(initial_value=0.0,name='var')
sess1 = tf.Session(server1.target)
sess2 = tf.Session(server2.target)
sess1.run(tf.global_variables_initializer())
sess2.run(tf.global_variables_initializer())
print("Initial value of var in session 1:", sess1.run(var))
print("Initial value of var in session 2:", sess2.run(var))
sess1.run(var.assign_add(1.0))
print("Incremented var in session 1")
print("Value of var in session 1:", sess1.run(var))
print("Value of var in session 2:", sess2.run(var))

Initial value of var in session 1: 0.0
Initial value of var in session 2: 0.0
Incremented var in session 1
Value of var in session 1: 1.0
Value of var in session 2: 1.0


#### 存放

In [9]:
def run_with_location_trace(sess,op):
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op,options=run_options,run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
        print(device.device)
        for node in device.node_stats:
            print(' ',node.node_name)

In [10]:
run_with_location_trace(sess1,var)

/job:local/replica:0/task:0/device:CPU:0
  _SOURCE
/job:local/replica:0/task:0/device:GPU:0
  _SOURCE
  var


In [11]:
run_with_location_trace(sess1,var.assign_add(1.0))

/job:local/replica:0/task:0/device:GPU:0
  _SOURCE
  var
  AssignAdd_1/value
  AssignAdd_1
/job:local/replica:0/task:0/device:CPU:0
  _SOURCE


In [12]:
run_with_location_trace(sess2,var)

/job:local/replica:0/task:1/device:CPU:0
  RecvTensor
/job:local/replica:0/task:0/device:GPU:0
  _SOURCE
  var
/job:local/replica:0/task:1/device:CPU:0
  _SOURCE


In [13]:
# 要将变量或操作固定到特定任务上，使用tf.device
with tf.device('/job:local/task:0'):
    var1 = tf.Variable(0.0,name='var1')
with tf.device('/job:local/task:1'):
    var2 = tf.Variable(0.0,name='var2')
# ( This will initialize both variables)
sess1.run(tf.global_variables_initializer())

In [14]:
run_with_location_trace(sess1,var1)

/job:local/replica:0/task:0/device:CPU:0
  _SOURCE
/job:local/replica:0/task:0/device:GPU:0
  _SOURCE
  var1


In [15]:
# 即使我们尝试使用连接到第一个任务的会话来评估它，它仍然在第二个任务上运行
run_with_location_trace(sess1,var2)

/job:local/replica:0/task:0/device:CPU:0
  RecvTensor
/job:local/replica:0/task:1/device:GPU:0
  _SOURCE
  var2
/job:local/replica:0/task:0/device:CPU:0
  _SOURCE


In [16]:
run_with_location_trace(sess2,var1)

/job:local/replica:0/task:1/device:CPU:0
  RecvTensor
/job:local/replica:0/task:0/device:GPU:0
  _SOURCE
  var1
/job:local/replica:0/task:1/device:CPU:0
  _SOURCE


#### 计算图

In [17]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2224", "localhost:2225"]})
server1 = tf.train.Server(cluster,job_name='local',task_index=0)
server2 = tf.train.Server(cluster,job_name='local',task_index=1)

In [18]:
graph1 = tf.Graph()
with graph1.as_default():
    var1 = tf.Variable(0.0,name='var')
sess1 = tf.Session(target=server1.target,graph=graph1)
print(graph1.get_operations())

[<tf.Operation 'var/initial_value' type=Const>, <tf.Operation 'var' type=VariableV2>, <tf.Operation 'var/Assign' type=Assign>, <tf.Operation 'var/read' type=Identity>]


In [20]:
graph2 = tf.Graph()
sess2 = tf.Session(target=server2.target,graph=graph2)
print(graph2.get_operations())

[]


In [21]:
# 要访问共享变量，必须手动添加一个同名的变量到第二个图中
with graph2.as_default():
    var2 = tf.Variable(0.0,name='var')
sess1.run(var1.assign(1.0))
sess2.run(var2)

1.0